In [ ]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm

def preprocess_dataset(image_dir='data/images', csv_file='data/info.csv', output_dir='preprocessed', target_size=(128, 128)):
    # Load dataset info
    df = pd.read_csv(csv_file)

    # Filter relevant images: 'palmar left' or 'palmar right'
    filtered_df = df[df['aspectOfHand'].isin(['palmar left', 'palmar right'])]

    # Create output directory if not exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Resize and save images
    for _, row in tqdm(filtered_df.iterrows(), desc='transforming', total=filtered_df.shape[0]):
        img_name = row['imageName']
        img_path = os.path.join(image_dir, img_name)
        img = Image.open(img_path).convert("RGB")  # Ensure 24-bit color
        
        # Resize image to target size
        img_resized = img.resize(target_size)
        
        # Save preprocessed image
        img_resized.save(os.path.join(output_dir, img_name))

    print(f"Preprocessed dataset saved in {output_dir}")

# Run preprocessing
preprocess_dataset()


In [ ]:
# Preprocessing the images and create a ZIP archive for training
!python stylegan2-ada-pytorch/dataset_tool.py --source=preprocessed --dest=palm_dataset.zip

In [ ]:
!python stylegan2-ada-pytorch/train.py --outdir=training-runs --data=palm_dataset.zip --gpus=8 --cfg=stylegan2 --aug=ada --metrics=fid50k_full

In [ ]:
import torch
import numpy as np
import os
from PIL import Image
from typing import Optional

def generate_palm_images(output_dir: str, num_images: int, model_path: str, 
                         age: Optional[int] = None, gender: Optional[str] = None,
                         skin_color: Optional[str] = None, accessories: Optional[int] = None, 
                         side: Optional[str] = None):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load the trained StyleGAN2-ADA model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
        G = torch.load(model_path).to(device)  # Load generator model
    
    # Set random seed for reproducibility
    torch.manual_seed(42)
    
    for i in range(num_images):
        z = torch.randn(1, G.z_dim).to(device)  # Latent vector

        # Apply conditioning based on optional parameters (age, gender, skin color, etc.)
        conditioning = {
            'age': age,
            'gender': gender,
            'skin_color': skin_color,
            'accessories': accessories,
            'side': side
        }
        # Assume that the generator is conditioned on these attributes, map them to latent codes or style inputs
        # You need to design the latent manipulation based on your specific conditioning logic.

        img = G(z, conditioning)  # Generate an image

        # Convert image to PIL format and save
        img = (img.clamp(-1, 1).cpu().numpy() * 127.5 + 127.5).astype(np.uint8)
        img = Image.fromarray(img[0].transpose(1, 2, 0))  # Convert from CHW to HWC
        img.save(os.path.join(output_dir, f"palm_{i+1}.png"))

    print(f"Generated {num_images} images in {output_dir}")

# Example usage
generate_palm_images(output_dir='generated_palm_images', num_images=10, model_path='path/to/your_trained_model.pkl', 
                     age=30, gender='male', skin_color='medium', accessories=1, side='left')
